In [155]:
import matplotlib.pyplot as plt
# Turn Interactive Mode off. Only displays plot with plt.show()
plt.ioff()

import pandas as pd
import numpy as np
import json
import os

from collections import defaultdict

In [156]:
DATA_PATH = "./../scripts/output"

In [157]:
lambda_version = "step-v3"

# # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
stages_id = ["app"]
stages_suffix = [""]
powerValues = [1024, 2048, 4096, 8192]


# # Manual : crop->scaledown->mirror->bw->rotate->watermark
step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror", "_bw", "_rot", "_watermarked"]
powerValues = [1024, 2048, 4096, 8192]


# # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror_bw_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror_bw", "_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized_mirror_bw_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# Stage Wise Plots

In [151]:
def plotStageInvocationTimeVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    plt.figure(figsize=(10,5))
    for stage in stages_id: 
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        power_values = []
        stage_stats = []
        for stat in data['stats']:
            stage_stats.append(stat['averageDuration']/1000)
            power_values.append(stat['value'])
        plt.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    plt.legend()
    plt.title(step_version)
    plt.xlabel('Memory(MB)')
    plt.ylabel('Invocation Time(s)')

    # Create names on the x axis
    plt.xticks(power_values)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/InvocationTimeMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()
    
def plotStageInvocationCostVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    plt.figure(figsize=(10,5))
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        stage_stats = []
        power_values = []
        for stat in data['stats']:
            stage_stats.append(stat['averagePrice']/1000)
            power_values.append(stat['value'])
        plt.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    plt.legend()
    plt.title(step_version)
    plt.xlabel('Memory(MB)')
    plt.ylabel('Invocation Cost(USD)')

    # Create names on the x axis
    plt.xticks(power_values)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/InvocationCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()
    
    
def plotStageInvocationTimeCostVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(10,5))
    
    for stage in stages_id: 
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        power_values = []
        stage_stats = []
        for stat in data['stats']:
            stage_stats.append(stat['averageDuration']/1000)
            power_values.append(stat['value'])
        ax.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    ax.set_xlabel('Memory(MB)')
    ax.set_ylabel('Invocation Time(s)')
        
    ax2=ax.twinx()
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        stage_stats = []
        power_values = []
        for stat in data['stats']:
            stage_stats.append(stat['averagePrice']/1000)
            power_values.append(stat['value'])
        ax2.plot(power_values, stage_stats, label = stage, marker='x', linestyle="--")
        f.close()
    ax2.set_xlabel('Memory(MB)')
    ax2.set_ylabel('Invocation Cost(USD)')

    # Create names on the x axis
    plt.xticks(power_values)
    plt.legend()
    plt.title(step_version)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/TimeCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()

In [152]:
batch_len = 150
for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)
    # # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
    stages_id = ["app"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # Manual : crop->scaledown->mirror->bw->rotate->watermark
    step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
    step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
    step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

# Cumulative Time Cost Plots for Pipeline Variants

In [186]:
def getPipelineTimeCost(path, batch_name, step_version, stages_id, displayPlot=False):
    total_cost_dict = defaultdict(lambda : 0)
    total_time_dict = defaultdict(lambda : 0)
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        for stat in data['stats']:
            total_cost_dict[stat["value"]] += stat["averagePrice"] 
            total_time_dict[stat["value"]] += stat['averageDuration']/1000 
        f.close()
    return list(total_cost_dict.keys()), list(total_time_dict.values()), list(total_cost_dict.values())

def plotPipelineTimeCost(path, batch_name, step_version, stages_id, displayPlot=False):
    power_values, total_time, total_cost = getPipelineTimeCost(path, batch_name, step_version, stages_id)
    
    # Plot Time
    fig,ax = plt.subplots(figsize=(10,5))
    ax.plot(power_values, total_time, label="Invocation Time")
    ax.set_xlabel('Memory(MB)')
    ax.set_ylabel('Invocation Time(s)')
    plt.legend()
    
    # Plot Cost
    ax2=ax.twinx()
    ax2.plot(power_values, total_cost, linestyle="--", label="Invocation Cost")
    ax2.set_ylabel('Invocation Cost(USD)')
    
    # Label
    plt.xticks(power_values)
    plt.legend()
    plt.title(step_version)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/cumulative".format(batch_name)):
        os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/cumulative".format(batch_name))
    plt.savefig("./plots/{}/cumulative/CumulativeTimeCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()

### Plot

In [187]:
for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)

    # # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
    stages_id = ["app"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # Manual : crop->scaledown->mirror->bw->rotate->watermark
    step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
    step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
    step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

### Get Data

In [188]:
stagesList = [
    ["app"],
    ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"],
    ["crop", "scaledown", "mirror_bw_rotate_watermark"],
    ["crop", "scaledown", "mirror_bw", "rotate_watermark"],
    ["crop", "scaledown_mirror_bw_rotate_watermark"]
]


allStatsDict = {"pipeline":[], "aggregation":[], "batchLen":[], "memory":[], "timePerBatch":[], "costPerBatch":[]}

for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)
    pipelines = [
        "step-agg-csmbrw-{}Sec-batch".format(batch_len), 
        "step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len), 
        "step-agg-c-s-mbrw-{}Sec-batch".format(batch_len), 
        "step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len), 
        "step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    ]
    
    for step_pipeline, stages_id in zip(pipelines, stagesList):
        agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
        # Get Stats
        power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
        n = len(power_values)
        allStatsDict["pipeline"] += [step_pipeline]*(n+2)
        allStatsDict["aggregation"] += [agg_strat]*(n+2)
        allStatsDict["batchLen"] += [batch_len]*(n+2)
        
        # Add Stats for all Memory Values
        allStatsDict["memory"] += power_values
        allStatsDict["timePerBatch"] += total_time
        allStatsDict["costPerBatch"] += total_cost
        
        # Add the best cost
        allStatsDict["memory"] += ["least_cost"]
        allStatsDict["timePerBatch"] += [total_time[total_cost.index(min(total_cost))]]
        allStatsDict["costPerBatch"] += [min(total_cost)]
        
        # Add the least time
        allStatsDict["memory"] += ["least_time"]
        allStatsDict["timePerBatch"] += [min(total_time)]
        allStatsDict["costPerBatch"] += [total_cost[total_time.index(min(total_time))]]

In [189]:
totalStatDF = pd.DataFrame(allStatsDict)

In [190]:
totalStatDF["noOfBatch"] = totalStatDF.apply(lambda x: 10*60/x["batchLen"], axis=1)
totalStatDF["timeTotal"] = totalStatDF.apply(lambda x: x["timePerBatch"]*x["noOfBatch"], axis=1)
totalStatDF["costTotal"] = totalStatDF.apply(lambda x: x["costPerBatch"]*x["noOfBatch"], axis=1)

totalStatDF.to_csv('./data/Pipeline-Memory-TimeCost.csv')

In [192]:
totalStatDF[totalStatDF["batchLen"]==120]

,pipeline,aggregation,batchLen,memory,timePerBatch,costPerBatch,noOfBatch,timeTotal,costTotal
0,step-agg-csmbrw-120Sec-batch,csmbrw,120,512,784.404546,0.006589,5.0,3922.022728,0.032945
1,step-agg-csmbrw-120Sec-batch,csmbrw,120,1024,386.522434,0.006494,5.0,1932.612172,0.032468
2,step-agg-csmbrw-120Sec-batch,csmbrw,120,2048,189.280761,0.006360,5.0,946.403806,0.031799
3,step-agg-csmbrw-120Sec-batch,csmbrw,120,4096,104.937922,0.007052,5.0,524.689608,0.035259
4,step-agg-csmbrw-120Sec-batch,csmbrw,120,8192,80.498194,0.010819,5.0,402.490969,0.054095
5,step-agg-csmbrw-120Sec-batch,csmbrw,120,least_cost,189.280761,0.006360,5.0,946.403806,0.031799
6,step-agg-csmbrw-120Sec-batch,csmbrw,120,least_time,80.498194,0.010819,5.0,402.490969,0.054095
7,step-agg-c-s-m-b-r-w-120Sec-batch,c-s-m-b-r-w,120,512,792.182834,0.006654,5.0,3960.914169,0.033272
8,step-agg-c-s-m-b-r-w-120Sec-batch,c-s-m-b-r-w,120,1024,390.970037,0.006568,5.0,1954.850186,0.032842
9,step-agg-c-s-m-b-r-w-120Sec-batch,c-s-m-b-r-w,120,2048,193.144840,0.006490,5.0,965.724200,0.032449
